In [10]:
import xgboost as xgb
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tensorflow import keras
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split

In [11]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx"
data = pd.read_excel(url)

print(data.head())
# print(data.info())
# print(data.describe())

     X1     X2     X3      X4   X5  X6   X7  X8     Y1     Y2
0  0.98  514.5  294.0  110.25  7.0   2  0.0   0  15.55  21.33
1  0.98  514.5  294.0  110.25  7.0   3  0.0   0  15.55  21.33
2  0.98  514.5  294.0  110.25  7.0   4  0.0   0  15.55  21.33
3  0.98  514.5  294.0  110.25  7.0   5  0.0   0  15.55  21.33
4  0.90  563.5  318.5  122.50  7.0   2  0.0   0  20.84  28.28


In [16]:
X = data.iloc[:, :-2]
y = data['Y1']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_test, y_train, y_test  = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")
mlflow.set_experiment("heat_load_prediction")
model_name = "heat-forecast-prod"

In [18]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost Regressor": xgb.XGBRegressor(n_estimators=100, random_state=42)
}
for name, model in models.items():
    with mlflow.start_run():

        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        mlflow.log_param("model_name", name)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        mlflow.sklearn.log_model(model, name = model_name, input_example = X_train_scaled)

2025/12/08 11:25:32 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run funny-hen-961 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/076e7dbb2bea4c78b4668d492fdb5e56
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


2025/12/08 11:25:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run debonair-flea-460 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/89370e334f3749249bc3024aced7f671
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


2025/12/08 11:25:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run colorful-asp-495 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/705a287cf4ff4058b546f07129c8f522
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


In [25]:
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_t  = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
train_ds = TensorDataset(X_train_t, y_train_t)

val_split = int(0.2 * len(train_ds))
train_subset, val_subset = random_split(train_ds, [len(train_ds)-val_split, val_split])

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32)

class RegressionNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

model = RegressionNN(X_train_scaled.shape[1])    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.counter = 0
        self.best_loss = float("inf")
        self.best_state = None

    def step(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_state = model.state_dict()
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

early_stopping = EarlyStopping(patience=20)

# --- Training Loop ---
with mlflow.start_run():
    for epoch in range(500):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                pred = model(xb)
                val_losses.append(criterion(pred, yb).item())
        val_loss = sum(val_losses) / len(val_losses)

        print(f"Epoch {epoch+1}, val_loss={val_loss:.4f}")

        # Early stopping check
        if early_stopping.step(val_loss, model):
            print("Early stopping triggered!")
            break

    # Restore best weights
    model.load_state_dict(early_stopping.best_state)

    # Predict on test data
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_t).numpy()

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_param("model_name", "Pytorch NN")
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)

    # Save model
    mlflow.pytorch.log_model(model, name = model_name, input_example = X_train_scaled.astype("float32"))

Epoch 1, val_loss=549.8357
Epoch 2, val_loss=516.3857
Epoch 3, val_loss=461.0833
Epoch 4, val_loss=380.1158
Epoch 5, val_loss=277.0799
Epoch 6, val_loss=168.3986
Epoch 7, val_loss=82.8788
Epoch 8, val_loss=42.3065
Epoch 9, val_loss=32.1516
Epoch 10, val_loss=27.7435
Epoch 11, val_loss=24.7876
Epoch 12, val_loss=22.7668
Epoch 13, val_loss=21.6246
Epoch 14, val_loss=20.9114
Epoch 15, val_loss=19.9821
Epoch 16, val_loss=19.2694
Epoch 17, val_loss=18.6336
Epoch 18, val_loss=18.1524
Epoch 19, val_loss=17.5056
Epoch 20, val_loss=16.9990
Epoch 21, val_loss=16.8271
Epoch 22, val_loss=15.8893
Epoch 23, val_loss=15.7215
Epoch 24, val_loss=15.4434
Epoch 25, val_loss=14.7488
Epoch 26, val_loss=14.4586
Epoch 27, val_loss=14.1273
Epoch 28, val_loss=13.7918
Epoch 29, val_loss=13.3930
Epoch 30, val_loss=13.1360
Epoch 31, val_loss=12.9396
Epoch 32, val_loss=12.5319
Epoch 33, val_loss=12.3250
Epoch 34, val_loss=11.7853
Epoch 35, val_loss=11.7373
Epoch 36, val_loss=11.6139
Epoch 37, val_loss=11.2994
Epoc

2025/12/08 11:32:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run orderly-midge-973 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/67ee5baf140442c79e333a1b6e1b688b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


In [26]:
model = keras.Sequential([
    keras.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1)   # regression output
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="mse"
)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

with mlflow.start_run():
    model.fit(X_train_scaled, y_train, epochs=500, batch_size=32, validation_split=0.2, verbose = 1, callbacks=[early_stopping])
    y_pred = model.predict(X_test_scaled)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_param("model_name", "Neural Networks")
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)
    mlflow.sklearn.log_model(model, name = model_name, input_example = X_train_scaled)

2025-12-08 11:33:24.111259: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 593.9142 - val_loss: 514.0627
Epoch 2/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 549.7261 - val_loss: 467.9417
Epoch 3/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 494.0899 - val_loss: 408.0708
Epoch 4/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 421.3989 - val_loss: 330.7395
Epoch 5/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 330.2480 - val_loss: 239.9634
Epoch 6/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 226.0311 - val_loss: 148.9106
Epoch 7/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 132.4970 - val_loss: 79.3788
Epoch 8/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 68.3092 - val_loss: 48.5525
Epoch 9/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 43.3613 - val_loss: 39.4992
Epoch 10/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.7573 - val_loss: 34.5419
Epoch 11/500
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31.3463 - val_loss: 30.0991
Epoch 12/500
16/16 ━━━━━━━

2025/12/08 11:34:17 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
🏃 View run persistent-shad-523 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/6427899293a74a329c87699426b2703c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


In [28]:
# Register model
run_id = input('Run ID:')
model_uri = f'runs:/{run_id}/{model_name}'
with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Run ID: 6427899293a74a329c87699426b2703c


Registered model 'heat-forecast-prod' already exists. Creating a new version of this model...
2025/12/08 11:35:58 WARNING mlflow.tracking._model_registry.fluent: Run with id 6427899293a74a329c87699426b2703c has no artifacts at artifact path 'heat-forecast-prod', registering model based on models:/m-23fa2c90c13640d0812a0a7ee0cbadba instead
2025/12/08 11:35:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: heat-forecast-prod, version 4
Created version '4' of model 'heat-forecast-prod'.


🏃 View run persistent-shad-523 at: http://127.0.0.1:5000/#/experiments/195097301821250000/runs/6427899293a74a329c87699426b2703c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195097301821250000


In [33]:
model_uri = f"models:/{model_name}@champion"
loaded_model = mlflow.sklearn.load_model(model_uri)
y_pred = loaded_model.predict(X_test_scaled)
print("Prediction:", end="")
print(y_pred[:4])
print("Test:", end="")
print(y_test[:4])

Prediction:[15.09619  13.24421  32.464813 41.37811 ]
Test:[16.47 13.17 32.82 41.32]
